In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, circuit, extensions
import matplotlib.pyplot as plt
import math
import numpy as np
from functools import reduce
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
def normalize(l):
    normalization_factor = math.sqrt(sum([i ** 2 for i in l]))
    return [i / normalization_factor for i in l]

def init_gate(init_states, label=None, size=None):
    if not size:
        size = math.ceil(math.log2(len(init_states)))
    state_vector = normalize(init_states) + [0] * ((2 ** size) - len(init_states))
    #I know this is way ugly, but it's the easiest way to actualy turn initialize() into a gate
    return extensions.Initialize(state_vector).gates_to_uncompute().inverse().to_gate(label=label if label else None)

In [14]:
def x_indices(i, bits):
    indices = []
    for j in range(bits):
        bit = (i >> j) & 1
        if not bit:
            indices.append(j)
    return indices

In [15]:
def construct_oracle(data, labels):
    #[[0,1,5,7],[1,2,4,7]], [0,1]
    # gate = init_gate([0,2,10], "psi_0")
    data_bits = math.ceil(math.log2(data.shape[1]))
    counting_bits = math.ceil(math.log2(data.shape[0]))
    print(len(data), data.shape[0])
    q = QuantumRegister(data_bits + counting_bits + 1)
    qc = QuantumCircuit(q)

#     data = [[x,x*2,x**2] for x in range(1, 16)]
    qc.h(q[data_bits + 1:])
    for i in range(len(data)):
        x_i = [data_bits + 1 + x_i for x_i in x_indices(i,counting_bits)]
        if len(x_i) > 0:
            qc.x(x_i)
        gate = init_gate(data[i], f"psi_{i}", data_bits).control(counting_bits)
        qc.append(gate, q[data_bits + 1:] + q[:data_bits])
        if labels[i] == 1:
            x_gate = circuit.library.XGate().control(counting_bits)
            qc.append(x_gate, q[data_bits + 1:] + [q[data_bits]])

        if len(x_i) > 0:
            qc.x(x_i)
    return qc, data_bits, counting_bits

In [72]:
def destructive_swap(n):
    q = QuantumRegister(n * 2)
    circuit = QuantumCircuit(q)
    
    for i in range(n):
        circuit.cx(i, n+i)
        circuit.h(i)
#     print(circuit)
    return circuit.to_gate(label="destructive_swap")


In [76]:
def swap(n):
    q = QuantumRegister(n * 2 + 1)
    circuit = QuantumCircuit(q)
    circuit.h(q[0])
    
    for i in range(0, n):
        circuit.cswap(0, 1 + i, 1 + n+i)
    circuit.h(q[0])
    return circuit.to_gate(label="swap")

         ┌───┐      ┌───┐
q4846_0: ┤ H ├─■──■─┤ H ├
         └───┘ │  │ └───┘
q4846_1: ──────X──┼──────
               │  │      
q4846_2: ──────┼──X──────
               │  │      
q4846_3: ──────X──┼──────
                  │      
q4846_4: ─────────X──────
                         


In [17]:
def interpret_des_swap(counts):
    successes = 0
    for k in counts:
        #success if and of measurement has even parity, that's what the paper said
        success = reduce(lambda b, p: (p == ('1', '1')) ^ b , zip(list(k[:len(k)//2]), list(k[len(k)//2:])), True)
#         print(k, success)
#         success = k.count('1') % 2 == 0
        if success:
            successes += counts[k]
    if sum(counts.values()) == 0:
        return 0
    return  2 * (successes / sum(counts.values())) - 1

In [18]:
def construct_classifier(data, labels):
    input_size = math.ceil(math.log2(data.shape[1]))
    oracle, data_bits, counting_bits = construct_oracle(data, labels)
    print(oracle)
    oracle = oracle.to_gate(label="oracle")
    q = QuantumRegister(input_size + data_bits + counting_bits + 1)
    c = ClassicalRegister(input_size * 2 + 1)
    qc = QuantumCircuit(q, c)
    
    qc.append(oracle, q[input_size:])
#     print(input_size)
    qc.append(destructive_swap(input_size), q[:input_size] + q[input_size:input_size+input_size])
    for i in range(input_size * 2 + 1):
        qc.measure(i,i)
    return qc


In [176]:
df = pd.read_csv("bezdekIris.data", header=None)

In [177]:
df[4] = [0 if p == "Iris-setosa" else 1 if p == "Iris-virginica" else 2 for p in df[4]]
df = df[df[4] != 0]

In [178]:
train, test = train_test_split(df, test_size=0.36)
data = train[[0,1]].to_numpy()
labels = train[4].to_numpy()

In [179]:
print(labels)

[2 1 1 1 2 2 2 2 2 1 2 1 1 2 2 1 1 1 1 1 2 2 2 1 1 2 1 1 1 2 1 2 2 2 2 2 2
 1 2 1 1 1 2 2 2 2 2 1 1 2 2 1 1 2 1 2 2 2 1 2 1 1 1 2]


In [67]:
# data = np.ones((20,4))
# data = np.array([np.array([1,2,0,4]) for i in range(20)])


qc_classifier = construct_classifier(data, labels)


64 64
                   ┌───────┐          ┌───────┐               ┌───────┐     »
q3828_0: ──────────┤ psi_0 ├──────────┤ psi_1 ├───────────────┤ psi_2 ├─────»
                   └───┬───┘          └───┬───┘┌───┐          └───┬───┘┌───┐»
q3828_1: ──────────────┼──────────────────┼────┤ X ├──────────────┼────┤ X ├»
         ┌───┐┌───┐    │    ┌───┐         │    └─┬─┘┌───┐         │    └─┬─┘»
q3828_2: ┤ H ├┤ X ├────■────┤ X ├─────────■──────■──┤ X ├─────────■──────■──»
         ├───┤├───┤    │    ├───┤┌───┐    │      │  ├───┤         │      │  »
q3828_3: ┤ H ├┤ X ├────■────┤ X ├┤ X ├────■──────■──┤ X ├─────────■──────■──»
         ├───┤├───┤    │    ├───┤├───┤    │      │  ├───┤┌───┐    │      │  »
q3828_4: ┤ H ├┤ X ├────■────┤ X ├┤ X ├────■──────■──┤ X ├┤ X ├────■──────■──»
         ├───┤├───┤    │    ├───┤├───┤    │      │  ├───┤├───┤    │      │  »
q3828_5: ┤ H ├┤ X ├────■────┤ X ├┤ X ├────■──────■──┤ X ├┤ X ├────■──────■──»
         ├───┤├───┤    │    ├───┤├───┤    │      │  ├───┤├

In [69]:
def classify(inp):
    num_qubits = qc_classifier.num_qubits

    data_size = math.ceil(math.log2(data.shape[1]))

    num_cbits = data_size * 2 + 1


    q = QuantumRegister(num_qubits)
    c = ClassicalRegister(num_cbits)
    qc = QuantumCircuit(q, c)

    # inp_gate = init_gate([1,2,0,4])
    inp_gate = init_gate(inp, label="input", size=data_size)


    qc.append(inp_gate, q[0:data_size])
    qc = qc.compose(qc_classifier)
    backend = Aer.get_backend('qasm_simulator')
    print("starting")
    job = execute(backend=backend, experiments=qc, shots=100)
    result = job.result()
    print("done")
    
    counts = result.get_counts()
    counts_0 = {key[1:]:value for key, value in counts.items() if key[0] == '0'}
    counts_1 = {key[1:]:value for key, value in counts.items() if key[0] == '1'}

    return (interpret_des_swap(counts_0), interpret_des_swap(counts_1))
    

In [152]:
label_names = [0,1]
correct = 0
count = 0
for index, row in test.iterrows():
    label = row[4]
    results = classify(row[[0,1]].tolist())
    cor = False
    if results[0] > results[1] and label == label_names[0]:
        correct += 1
        cor = True
    elif results[0] < results[1] and label == label_names[1]:
        correct += 1
        cor = True
    count += 1
    print(f"label:{label}, result:{label_names[0]}:{results[0]}, {label_names[1]}:{results[1]}, {'correct' if cor else 'incorrect'}, accuracy:{correct / count}")

print(correct / len(test))


starting


KeyboardInterrupt: 

In [ ]:
num_qubits = qc_classifier.num_qubits

data_size = math.ceil(math.log2(data.shape[1]))

num_cbits = data_size * 2 + 1


q = QuantumRegister(num_qubits)
c = ClassicalRegister(num_cbits)
qc = QuantumCircuit(q, c)

# inp_gate = init_gate([1,2,0,4])
inp_gate = init_gate([1,2,0,4], label="input", size=data_size)


qc.append(inp_gate, q[0:2])
qc = qc.compose(qc_classifier)
print(qc)
backend = Aer.get_backend('qasm_simulator')
job = execute(backend=backend, experiments=qc, shots=2000)
result = job.result()

# print(result.get_counts())
counts = result.get_counts()

counts_0 = {key[1:]:value for key, value in counts.items() if key[0] == '0'}
counts_1 = {key[1:]:value for key, value in counts.items() if key[0] == '1'}

print(counts)
print(interpret_des_swap(counts_0))
print(interpret_des_swap(counts_1))

In [180]:
def construct_classifier_swap(data, labels):
    input_size = math.ceil(math.log2(data.shape[1]))
    oracle, data_bits, counting_bits = construct_oracle(data, labels)
#     print(oracle)
    oracle = oracle.to_gate(label="oracle")
    q = QuantumRegister(input_size + data_bits + counting_bits + 2)
    c = ClassicalRegister(2)
    qc = QuantumCircuit(q, c)
    
    qc.append(oracle, q[input_size + 1:])
#     print(input_size)
    qc.append(swap(input_size), q[0:1] + q[1:input_size + 1] + q[input_size + 1:input_size+input_size + 1])
    qc.measure(q[0], c[0])
    qc.measure(q[2 * input_size + 1], c[1])
    
    return qc



In [181]:
qc_classifier_swap = construct_classifier_swap(data, labels)

64 64
          ┌───┐   ┌───┐
q15395_0: ┤ H ├─■─┤ H ├
          └───┘ │ └───┘
q15395_1: ──────X──────
                │      
q15395_2: ──────X──────
                       


In [172]:
def interpret_swap(counts):
    try:
        return 2 * counts['0'] / sum(counts.values()) - 1
    except:
        return 0

In [173]:
def classify_swap(inp):
    num_qubits = qc_classifier_swap.num_qubits

    data_size = math.ceil(math.log2(data.shape[1]))

    num_cbits = 2


    q = QuantumRegister(num_qubits)
    c = ClassicalRegister(num_cbits)
    qc = QuantumCircuit(q, c)

    # inp_gate = init_gate([1,2,0,4])
    inp_gate = init_gate(inp, label="input", size=data_size)


    qc.append(inp_gate, q[1:data_size + 1])
    qc = qc.compose(qc_classifier_swap)
    backend = Aer.get_backend('qasm_simulator')
    job = execute(backend=backend, experiments=qc, shots=1000)
    result = job.result()
    
    counts = result.get_counts()
    counts_0 = {key[1:]:value for key, value in counts.items() if key[0] == '0'}
    counts_1 = {key[1:]:value for key, value in counts.items() if key[0] == '1'}
#     print(counts, counts_0, counts_1)
#     print(counts)
    return (interpret_swap(counts_0), interpret_swap(counts_1))

In [183]:
label_names = [2,1]
correct = 0
count = 0
for index, row in test.iterrows():
    label = row[4]
    results = classify_swap(row[[0,1]].tolist())
    cor = False
    if results[0] > results[1] and label == label_names[0]:
        correct += 1
        cor = True
    elif results[0] < results[1] and label == label_names[1]:
        correct += 1
        cor = True
    count += 1
    print(f"label:{label}, result:{label_names[0]}:{results[0]}, {label_names[1]}:{results[1]}, {'correct' if cor else 'incorrect'}, accuracy:{correct / count}")

print(correct / len(test))


label:2.0, result:2:0.9926199261992621, 1:0.9956331877729259, incorrect, accuracy:0.0
label:2.0, result:2:0.996309963099631, 1:0.9956331877729259, correct, accuracy:0.5
label:2.0, result:2:1.0, 1:0.9956521739130435, correct, accuracy:0.6666666666666666
label:1.0, result:2:0.9962894248608534, 1:1.0, correct, accuracy:0.75
label:2.0, result:2:1.0, 1:1.0, incorrect, accuracy:0.6
label:2.0, result:2:0.9925093632958801, 1:0.9914163090128756, correct, accuracy:0.6666666666666666
label:1.0, result:2:1.0, 1:1.0, incorrect, accuracy:0.5714285714285714
label:1.0, result:2:1.0, 1:1.0, incorrect, accuracy:0.5
label:2.0, result:2:1.0, 1:0.995850622406639, correct, accuracy:0.5555555555555556
label:1.0, result:2:0.9922928709055876, 1:1.0, correct, accuracy:0.6
label:1.0, result:2:1.0, 1:0.9956521739130435, incorrect, accuracy:0.5454545454545454
label:2.0, result:2:1.0, 1:0.9956043956043956, correct, accuracy:0.5833333333333334
label:1.0, result:2:0.99624765478424, 1:0.9957173447537473, incorrect, ac

In [175]:
input_size = 1
q = QuantumRegister(3)
c = ClassicalRegister(1)
qc = QuantumCircuit(q, c)

qc.h(q[1:3])
qc.ry(0.1, q[1])
qc.ry(0.9, q[2])

# qc.x(q[1])
#     print(input_size)
qc.barrier()
qc.h(q[0])

qc.cswap(0,1,2)
qc.h(q[0])

# qc.append(swap(input_size), q[0:1] + q[1:input_size + 1] + q[input_size + 1:input_size+input_size + 1])
qc.measure(q[0], c[0])

backend = Aer.get_backend('qasm_simulator')
job = execute(backend=backend, experiments=qc, shots=1000)
result = job.result()

counts = result.get_counts()
print(qc)
print(counts)

                           ░ ┌───┐   ┌───┐┌─┐
q15242_0: ─────────────────░─┤ H ├─■─┤ H ├┤M├
          ┌───┐┌─────────┐ ░ └───┘ │ └───┘└╥┘
q15242_1: ┤ H ├┤ RY(0.1) ├─░───────X───────╫─
          ├───┤├─────────┤ ░       │       ║ 
q15242_2: ┤ H ├┤ RY(0.9) ├─░───────X───────╫─
          └───┘└─────────┘ ░               ║ 
  c338: 1/═════════════════════════════════╩═
                                           0 
{'1': 83, '0': 917}
